# Escopo

* namespaces e escopos
    * exec e eval?
* naming and binding
    * contador de referencias e o comando del
    * modificações ao deletar (ou a eficiência do comando del)
    * singletons implícitos (chars e ints pequenos)
    * garbage collection
* Bonus: if name == main

## Nomes e vínculos

### Mecanismo de nomeação em Python

Quando atribuímos algum objeto a um **nome** em Python, usando o operador de atribuição ```=```, não estamos propriamente pegando o conteúdo atribuído e copiando para um local da memória reservado àquele nome. Ocorre o inverso: o Python faz com que aquele nome aponte para o local da memória onde está o conteúdo atribuído. Em outras palavras, a atribuição por si só não copia dados. Outro termo usado na documentação oficial para designar o nome é **identificador**.

Já o ponteiro que aponta para o objeto, ou seja, o valor que está  estocado na memória por trás deste nome, é chamado de **referência**.

O ato de estabelecer essa conexão entre nomes e referências a objetos no contexto de um namespace é chamado de **vínculo** ou vinculação.

(Obs.: em alguns textos pode-se encontrar uma diferença sutil entre identificador e nome, sendo o primeiro mais usado para se referir ao tipo de token na análise léxica, e o segundo empregado para o uso efetivo do identificador na memória, em tempo de execução.)

Ao atribuirmos um nome a outro nome já existente, como no código abaixo, o Python vincula o novo nome à mesma referência por trás do nome original. Assim, se atribuirmos um novo objeto ao nome original, estabelecemos para aquele nome um novo vínculo, que abandona a primeira referência. Notar que a função ```print()``` não usa o valor do nome em si (```'x'```) - este é usado apenas para passar a referência ao objeto subjacente.

In [184]:
x = 1 # x aponta para um objeto int(1)
y = x # y aponta para o mesmo objeto int(1)
x += 1 # como int é imutável, é instanciado um novo objeto int(2)
print(f'Valor apontado por x: {x}') # x passa a apontar para este novo objeto int(2)
print(f'Valor apontado por y: {y}') # y preserva a referência ao objeto original int(1)

Valor apontado por x: 2
Valor apontado por y: 1


 Notar, ainda, que o código acima funciona dessa forma porque um número inteiro é um imutável, levando o Python a criar uma nova instância de objeto ao realizarmos a atribuição ```+=```. O mesmo pode não ocorrer com objetos mutáveis. Vejamos a diferença no caso do código abaixo. Como estamos modificando o valor do objeto já atribuído, nenhuma nova instância é criada, preservando em ambos os identificadores a referência original.

In [ ]:
x = [1]
y = x
x += [1]
print(f'Valor apontado por x: {x}')
print(f'Valor apontado por y: {y}')

Valor apontado por x: [1, 1]
Valor apontado por y: [1, 1]


Dois nomes vinculados a um exato mesmo objeto na memória são por vezes chamados de **sinônimos**.

### Criação de vínculos

### Contador de vínculos e o comando ```del```

Todo valor em Python, quando é criado, recebe uma sequência de bits logo em seu começo que trata única e exclusivamente de contar o número de vínculos que atualmente levam àquele valor. Sempre que executamos a instrução ```del [nome]```, o contador de referências é decrescido em uma unidade. Sempre que adicionamos uma referência ou um vínculo, o contador é acrescido uma unidade.

Quando o contador de referências de um objeto chega a ```0```, ele não é mais encontrável no programa, pois não há como saber onde ele se encontra na memória. Tal objeto estará então pronto para ser coletado pelo coletor de lixo automático do Python (*garbage collector*).

** INTRODUZIR VICISSITUDES DO GARBAGE COLLECTOR **

No código abaixo, usamos as bibliotecas ```sys``` e ```ctypes``` para obter acesso direto ao conteúdo do objeto ```obj1``` na memória. Não nos preocuparemos aqui com os detalhes dessas funcionalidades, que não são tema deste tópico. Tudo que precisamos saber é que este código imprime os bits do objeto. Notar que o primeiro byte do objeto estoca o número 1 em binários (```00000001```), justamente porque existe um único vínculo a este objeto, envolvendo o nome ```obj1```.

In [165]:
import ctypes
import sys

obj1 = 'b' * 500 # cria o objeto (uma string grande)
addr = id(obj1) # pega o endereço do objeto na memória
n_bytes = sys.getsizeof(obj1) # pega o número de bytes ocupado pelo objeto
ptr = ctypes.cast(addr, ctypes.POINTER(ctypes.c_uint8)) # cria um pointer de C para o endereço do objeto
lista_bits = ''.join(f'{ptr[i]:08b} ' for i in range(n_bytes))[:-1].split(' ') # lê cada byte do objeto e estoca em uma lista de strings

# loop para visualizar n bytes em uma linha
n = 8
visualizacao = ''
espaco = '\n '
for i, byte in enumerate(lista_bits):
    tipo_de_espaco = bool((i+1) % n)
    visualizacao += (byte + espaco[tipo_de_espaco])

print(visualizacao)

00000001 00000000 00000000 00000000 00000000 00000000 00000000 00000000
01110000 10011111 10010101 11000111 11111001 01111111 00000000 00000000
11110100 00000001 00000000 00000000 00000000 00000000 00000000 00000000
01110110 01001001 00011001 11100000 00000010 10111000 10101101 10000001
01100101 00011000 10011000 11000111 11111001 01111111 00000000 00000000
01100010 01100010 01100010 01100010 01100010 01100010 01100010 01100010
01100010 01100010 01100010 01100010 01100010 01100010 01100010 01100010
01100010 01100010 01100010 01100010 01100010 01100010 01100010 01100010
01100010 01100010 01100010 01100010 01100010 01100010 01100010 01100010
01100010 01100010 01100010 01100010 01100010 01100010 01100010 01100010
01100010 01100010 01100010 01100010 01100010 01100010 01100010 01100010
01100010 01100010 01100010 01100010 01100010 01100010 01100010 01100010
01100010 01100010 01100010 01100010 01100010 01100010 01100010 01100010
01100010 01100010 01100010 01100010 01100010 01100010 01100010 0

Agora vamos encapsular uma parte deste código na função ```bits()```, que retorna os bits do objeto. Criamos também uma função de comparação para imprimir verticalmente o conteúdo de dois objetos de mesmo tamanho. O número de bytes impresso é customizável por meio do atributo ```bytes_mostrados```.

In [260]:
def bits(x):
    addr = id(x)
    n_bytes = sys.getsizeof(x)
    ptr = ctypes.cast(addr, ctypes.POINTER(ctypes.c_uint8))
    bits = ''.join(f'{ptr[i]:08b} ' for i in range(n_bytes))[:-1].split(' ')
    return bits

def comparar(x_bits, y_bits, bytes_mostrados=1):
    n = len(x_bits)
    if bytes_mostrados < n:
        n = bytes_mostrados
    print('   1        2       iguais?')
    for i in range(n):
        print(x_bits[i], y_bits[i], ' ', x_bits[i] == y_bits[i])

Em seguida, geramos alguns objetos e observamos seu conteúdo. No exemplo abaixo, o objeto ```obj2``` é nomeado e salvamos seu conteúdo em bits usando a função acima. Posteriormente, vinculamos o nome ```obj3``` ao mesmo objeto referenciado pelo nome ```obj2```, e de novo salvamos seu conteúdo em bits. Ao final, imprimimos apenas o primeiro byte de cada objeto (contador de referências).

In [261]:
obj2 = 'e' * 500
antes = bits(obj2)
obj3 = obj2
depois = bits(obj3)
comparar(antes, depois)

TypeError: 'str' object is not callable

Observa-se que o contador após o primeiro vínculo conta 2 (```00000010```) em vez de 1 (```00000001```)! Por que isso acontece? Porque, como explicado acima, passar um objeto como argumento de uma função também cria um vínculo a ele. Como agora temos nosso código encapsulado na função ```bits()```, o mesmo objeto que havia sido vinculado a ```obj2``` também passa a ser vinculado ao argumento ```x``` e sua contagem de vínculos passa a ser 2. Em seguida, ao definirmos o nome sinônimo ```obj3```, o contador de vínculos passa a contar 3 (```00000011```).

In [258]:
from itertools import permutations
from string import ascii_lowercase

letras = [letra for letra in ascii_lowercase[:17]]
print(letras)

obj4 = 1000
bits(obj4)
#for permutacao in permutations(letras, 2):
#    print(permutacao)
#    nome = permutacao[0] + permutacao[1]
#    print(nome)
#    exec(f'{nome} = obj4')

['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q']


TypeError: 'str' object is not callable

In [256]:
sys.getrefcount(obj4)
id(obj4)

TypeError: 'str' object is not callable

## 

## Bônus

### Operadores ```is``` vs. ```==```

Uma confusão comum entre iniciantes em Python é quando usar ```is``` e quando usar ```==``` para comparar objetos. A dúvida se resolve quando compreendemos o mecanismo de nomeação em Python: ```==``` compara se dois objetos potencilamente diferentes têm o mesmo valor, enquanto ```is``` compara se duas expressões resultam no exato mesmo objeto (ou seja, se as referências por trás de cada lado são as mesmas). Assim, ```x is y``` é equivalente a ```id(x) == id(y)```.

In [26]:
x = [1]
y = [1]
print(f'x e y são o mesmo objeto? {x is y}')
print(f'x e y têm o mesmo valor? {x == y}')


x e y são o mesmo objeto? False
x e y têm o mesmo valor? True


É claro que é mais eficiente comparar apenas referências em vez de comparar o objeto inteiro. Porém, vale atentar para algumas nuances no uso deste operador. Quando ambas as expressões retornam o mesmo objeto único (*singletons*) ou imortal, a comparação ```is``` sempre retorna ```True```. Isso pode parecer óbvio, mas às vezes não é fácil de reconhecer para iniciantes na linguagem. Por exemplo, você já se deparou com os casos abaixo?

In [226]:
# caso 1
x = 5
print('caso 1:', type(x) is int)

# caso 2
u = 1
v = 1
print('caso 2:', u is v)

# caso 3
y = 'a'
z = 'a'
print('caso 3:', y is z)

# caso 4
t500 = 'g' * 500
w500 = 'g' * 500
print('caso 4:', t500 is w500)

# caso 5
t5000 = 'h' * 5000
w5000 = 'h' * 5000
print('caso 5:', t5000 is w5000)

# caso 6
print('caso 6:', 10**9 is 10**9) # numeros grandes nao deveriam ser objetos identicos

caso 1: True
caso 2: True
caso 3: True
caso 4: True
caso 5: False
caso 6: True


<>:26: SyntaxWarning: "is" with 'int' literal. Did you mean "=="?
<>:26: SyntaxWarning: "is" with 'int' literal. Did you mean "=="?
C:\Users\henri\AppData\Local\Temp\ipykernel_76124\2384360586.py:26: SyntaxWarning: "is" with 'int' literal. Did you mean "=="?
  print('caso 6:', 10**9 is 10**9) # numeros grandes nao deveriam ser objetos identicos


O primeiro caso ocorre porque objetos do tipo ```type``` só admitem uma instância, logo o ```int``` retornado por ```type(int)``` é exatamente o mesmo objeto, na memória, do objeto único ```int```. Isso é possível porque, em Python, todo objeto sabe qual é seu tipo. Em CPython, isso é implementado por meio de um ponteiro ao singleton do tipo correspondente (informação que faz parte do conteúdo do objeto).

Já o segundo e o terceiro casos ocorrem porque, em CPython, números inteiros pequenos e expressões reduzidas recebem apenas um endereço de memória, que é apontado por todos os vínculos que usam tal valor (objetos imortais).

Por fim, o quarto caso ocorre porque, de novo na implementação CPython, podem ocorrer otimizações por parte do interpretador ao encontrar expressões que usam sucessivamente valores idênticos, principalmente dentro de uma mesma linha lógica, igualando duas referências. Notar que o Python avisa da inadequação de usar ```is``` para comparar literais.

A partir de Python 3.12, objetos imortais têm uma contagem de referências gigantesca (na implementação do meu computador e da minha versão, $2^{32} - 1$), que não pode ser diminuída, de modo que esses objetos jamais podem ser coletados pelo garbage collector. O mesmo ocorre com alguns singletons essenciais, tais como ```None```, ```True``` e ```False```, ou os tipos fundamentais da linguagem, como os objetos ```int``` e ```str```.

**ESCREVER SOBRE O QUINTO CASO E READEQUAR TEXTO E SUA POSIÇÃO**

In [205]:
class Uou:
    pass

sys.getrefcount(str)

4294967295

In [92]:
antes = bits(1)
x = 1
depois = bits(x)
comparar(antes, depois)

  obj 1    obj 2    iguais?
11111111 11111111   True
11111111 11111111   True
11111111 11111111   True
11111111 11111111   True
00000000 00000000   True
00000000 00000000   True
00000000 00000000   True
00000000 00000000   True
00010000 00010000   True
01101000 01101000   True
10010101 10010101   True
11000111 11000111   True
11111001 11111001   True
01111111 01111111   True
00000000 00000000   True
00000000 00000000   True
00001000 00001000   True
00000000 00000000   True
00000000 00000000   True
00000000 00000000   True
00000000 00000000   True
00000000 00000000   True
00000000 00000000   True
00000000 00000000   True
00000001 00000001   True
00000000 00000000   True
00000000 00000000   True
00000000 00000000   True


### Exceção à imutabilidade

listas dentro de tuplas

### Criando cópias de listas